In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

import xgboost as xgb

import pandas as pd

from helper import util_two_class

Using TensorFlow backend.


# 0. Get folder names for train and validation

In [2]:
folder_train = '/path/to/9_FINAL/data/machine_learning/two_class/distance_prepared/training_downsampled_sum/training_'

In [3]:
folder_validation = '/path/to/9_FINAL/data/machine_learning/two_class/distance_prepared/validation/cross_validation/sum/val_fold_'

In [4]:
# sanity check
validation_fold_0 = pd.read_csv(folder_validation+'0.csv',
                               sep=";")
validation_fold_0

,id,frequency,pidspread,pldspread,label,p1,p10,p11,p12a,p12b,...,p4,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d
0,343190981,1,1,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,453050043,1,1,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-8.0,0.0,0.0,0.0,0.0,0.0,0.0
2,344166346,2,1,2,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,240868865,17,3,16,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0
4,297345077,3,3,3,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1322851,203939908,1,1,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1322852,376449341,1,1,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0
1322853,28222046,2,1,2,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1322854,393022542,1,1,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# get labels
validation_fold_0.label.value_counts()

0    1300010
1      22846
Name: label, dtype: int64

In [6]:
# get goldstandard
goldstandard = pd.read_csv('/path/to/9_FINAL/data/goldstandard/goldstandard_distance_sum.csv',
                          sep=";")
goldstandard

,id,p1,p10,p12a,p12c,p15b,p16,p2,p20a,p20c,...,p3a,p4,p43,p5,p7,p8a,p8b,p8c,p8d,label
0,1231983,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,6969816,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,14220439,0.0,0.0,0.0,0.0,0.0,0.0,-9.0,0.0,0.0,...,-3.0,0.0,0.0,-9.0,0.0,0.0,0.0,0.0,0.0,0
3,14431636,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,19194895,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,499955920,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0
135,501384948,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0
136,507206333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
137,508437025,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [7]:
# get frequency, pidspread and pldspread
meta_gold = pd.read_csv('/path/to/9_FINAL/data/goldstandard/goldstandard_all.csv',
                        sep=";", usecols=['_id', 'frequency', 'pidspread', 'pldspread'])
meta_gold.columns = ['id', 'frequency', 'pidspread', 'pldspread']
meta_gold

,id,frequency,pidspread,pldspread
0,141663461,3,2,3
1,147713771,3,2,3
2,270852864,1,1,1
3,516170918,1,1,1
4,387574181,1,1,1
...,...,...,...,...
134,420639686,2,1,2
135,199660983,3,3,3
136,218064825,5,2,5
137,89573263,2,1,2


In [8]:
# merge data to goldstandard
goldstandard = pd.merge(meta_gold, goldstandard)
goldstandard

,id,frequency,pidspread,pldspread,p1,p10,p12a,p12c,p15b,p16,...,p3a,p4,p43,p5,p7,p8a,p8b,p8c,p8d,label
0,141663461,3,2,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0
1,147713771,3,2,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,270852864,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,516170918,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1
4,387574181,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,420639686,2,1,2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0,0
135,199660983,3,3,3,15.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-14.0,15.0,0.0,0.0,0.0,0.0,0
136,218064825,5,2,5,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
137,89573263,2,1,2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0


In [9]:
# get pid column with 0 for the columns without values
pad_with_zeros = [0] * len(goldstandard)

In [10]:
# get difference
difference = list(set(validation_fold_0.columns.to_list()) - set(goldstandard.columns.to_list()))
difference.sort()
difference

['p11',
 'p12b',
 'p13',
 'p14',
 'p15a',
 'p20b',
 'p20d',
 'p21b',
 'p22a',
 'p22b',
 'p29a',
 'p29c',
 'p30a',
 'p30b',
 'p31a',
 'p31b',
 'p37',
 'p38',
 'p39',
 'p42',
 'p6']

In [11]:
len(difference)

21

In [12]:
# delete label column from validation
del validation_fold_0['label']

In [13]:
# insert columns
for element in difference:
    goldstandard.insert(validation_fold_0.columns.get_loc(element), element, pad_with_zeros)
goldstandard.columns

Index(['id', 'frequency', 'pidspread', 'pldspread', 'p1', 'p10', 'p11', 'p12a',
       'p12b', 'p12c', 'p13', 'p14', 'p15a', 'p15b', 'p16', 'p2', 'p20a',
       'p20b', 'p20c', 'p20d', 'p21a', 'p21b', 'p21c', 'p21d', 'p22a', 'p22b',
       'p23a', 'p23b', 'p23c', 'p23d', 'p23e', 'p24', 'p25', 'p26', 'p27a',
       'p27b', 'p28a', 'p28b', 'p28c', 'p28d', 'p29a', 'p29c', 'p30a', 'p30b',
       'p31a', 'p31b', 'p34', 'p36', 'p37', 'p38', 'p39', 'p3a', 'p4', 'p42',
       'p43', 'p5', 'p6', 'p7', 'p8a', 'p8b', 'p8c', 'p8d', 'label'],
      dtype='object')

In [14]:
validation_fold_0.columns

Index(['id', 'frequency', 'pidspread', 'pldspread', 'p1', 'p10', 'p11', 'p12a',
       'p12b', 'p12c', 'p13', 'p14', 'p15a', 'p15b', 'p16', 'p2', 'p20a',
       'p20b', 'p20c', 'p20d', 'p21a', 'p21b', 'p21c', 'p21d', 'p22a', 'p22b',
       'p23a', 'p23b', 'p23c', 'p23d', 'p23e', 'p24', 'p25', 'p26', 'p27a',
       'p27b', 'p28a', 'p28b', 'p28c', 'p28d', 'p29a', 'p29c', 'p30a', 'p30b',
       'p31a', 'p31b', 'p34', 'p36', 'p37', 'p38', 'p39', 'p3a', 'p4', 'p42',
       'p43', 'p5', 'p6', 'p7', 'p8a', 'p8b', 'p8c', 'p8d'],
      dtype='object')

# 1. Logistic Regression

## 1.1 Distance columns

In [7]:
lr = LogisticRegression(random_state=42)

In [8]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['frequency', 'pidspread', 
                 'pldspread', 'id'],
                lr)

/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[854950 445060]
 [ 11740  11106]]
Precision 0.0243464002139572 and Recall 0.4861244856867723 and F1 0.04637044583434235
----------------------------------


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[801006 499004]
 [ 10449  12397]]
Precision 0.0242412509948162 and Recall 0.5426332837258163 and F1 0.04640924516188205
----------------------------------


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[943098 356912]
 [ 13443   9403]]
Precision 0.025669164516877552 and Recall 0.411581896174385 and F1 0.0483244723906044
----------------------------------


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[798041 501969]
 [ 10414  12432]]
Precision 0.024167915692232325 and Recall 0.5441652805742799 and F1 0.046280388722505665
----------------------------------


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[936076 363936]
 [ 13357   9492]]
Precision 0.025418554580802725 and Recall 0.4154229944417699 and F1 0.04790588401547402
----------------------------------
---------------------
Overall Precision: 0.0247686571997372
Overall Recall: 0.47998558812060466
Overall F1: 0.0470580872249617
---------------------
Negative Precision: 0.9865147644642696
Negative Recall: 0.6666363427814201
Negative F1: 0.7946058458206056


## 1.2 Scaled Distance columns

In [9]:
lr = LogisticRegression(random_state=42)

In [10]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['frequency', 'pidspread', 
                 'pldspread', 'id'],
                lr, scale=True)

[[     11 1299999]
 [      0   22846]]
Precision 0.01727035291360666 and Recall 1.0 and F1 0.03395430303093355
----------------------------------
[[1299737     273]
 [  22781      65]]
Precision 0.19230769230769232 and Recall 0.002845137004289591 and F1 0.005607315389924085
----------------------------------
[[1299970      40]
 [  22836      10]]
Precision 0.2 and Recall 0.00043771338527532174 and F1 0.0008735150244584207
----------------------------------
[[    161 1299849]
 [      3   22843]]
Precision 0.017270082528661246 and Recall 0.9998686859844174 and F1 0.033953704763447785
----------------------------------
[[    191 1299821]
 [      0   22849]]
Precision 0.017274906061224643 and Recall 1.0 and F1 0.03396310271352541
----------------------------------
---------------------
Overall Precision: 0.08882460676223698
Overall Recall: 0.6006303072747965
Overall F1: 0.02167038818445785
---------------------
Negative Precision: 0.9894437080466304
Negative Recall: 0.40000769220331484
Neg

## 1.3 Distance columns + frequencies

In [18]:
lr = LogisticRegression(random_state=42)

In [12]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['id'],
                lr)

/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[953753 346257]
 [ 12561  10285]]
Precision 0.02884653140443482 and Recall 0.4501882167556684 and F1 0.054218899912490646
----------------------------------


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[978223 321787]
 [ 13115   9731]]
Precision 0.02935285565188014 and Recall 0.4259388952114156 and F1 0.054920928762515385
----------------------------------


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[921205 378805]
 [ 12185  10661]]
Precision 0.027373377907185737 and Recall 0.4666462400420205 and F1 0.051713265682298844
----------------------------------


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[978607 321403]
 [ 13201   9645]]
Precision 0.02913474783113023 and Recall 0.4221745600980478 and F1 0.05450784698242977
----------------------------------


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[924207 375805]
 [ 12115  10734]]
Precision 0.02776951355490649 and Recall 0.4697798590747954 and F1 0.052439250784097245
----------------------------------
---------------------
Overall Precision: 0.02849540526990748
Overall Recall: 0.4469455542363895
Overall F1: 0.05356003842476638
---------------------
Negative Precision: 0.9868935999508803
Negative Recall: 0.7316856913648667
Negative F1: 0.8401975731318554


In [19]:
train = folder_train = pd.read_csv('/path/to/9_FINAL/data/machine_learning/two_class/distance_prepared/training_downsampled_sum/training_0.csv',
                                   sep=";", index_col=0)
train_new = train.drop(columns=['id'])
y_train = train_new['label'].to_list()
X_train = train_new.drop(columns=['label'])
X_train

,frequency,pidspread,pldspread,p1,p10,p11,p12a,p12b,p12c,p13,...,p4,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d
0,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
2,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2,1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-7.0,0.0,0.0,0.0,0.0,0.0,0.0
4,25,12,19,4.0,0.0,0.0,11.0,0.0,0.0,0.0,...,0.0,0.0,13.0,-7.0,0.0,0.0,9.0,9.0,9.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182769,7,4,7,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,2.0,0.0
182770,5,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-30.0,0.0,0.0,0.0,0.0,0.0,0.0
182771,4,1,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0
182772,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
goldstandard_new = goldstandard.drop(columns=['id'])
y_gold = goldstandard_new['label'].to_list()
X_gold = goldstandard_new.drop(columns=['label'])
X_gold

,frequency,pidspread,pldspread,p1,p10,p11,p12a,p12b,p12c,p13,...,p4,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d
0,3,2,3,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0.0,0,0.0,7.0,0.0,0.0,0.0
1,3,2,3,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
2,1,1,1,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
3,1,1,1,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0.0,2.0,0.0
4,1,1,1,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0.0,0,0.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,2,1,2,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0.0,0,0.0,17.0,0.0,0.0,0.0
135,3,3,3,15.0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,-14.0,0,15.0,0.0,0.0,0.0,0.0
136,5,2,5,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
137,2,1,2,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0.0,0,0.0,4.0,0.0,0.0,0.0


In [32]:
# only get positives for goldstandard
goldstandard[goldstandard.label == 1]

,id,frequency,pidspread,pldspread,p1,p10,p11,p12a,p12b,p12c,...,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d,label
3,516170918,1,1,1,0.0,0.0,0,0.0,0,0.0,...,0,0.0,0.0,0,0.0,0.0,0.0,2.0,0.0,1
16,105870566,3,3,3,0.0,0.0,0,0.0,0,0.0,...,0,0.0,-11.0,0,0.0,0.0,0.0,0.0,0.0,1
56,63424816,3,2,3,0.0,0.0,0,0.0,0,0.0,...,0,0.0,-16.0,0,0.0,0.0,0.0,0.0,0.0,1
93,139132257,9,3,9,0.0,0.0,0,0.0,0,0.0,...,0,0.0,-76.0,0,0.0,9.0,0.0,0.0,0.0,1
97,209693125,1,1,1,0.0,0.0,0,0.0,0,0.0,...,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,1


In [30]:
# get baseline for goldstandard
baseline = [0] * 139
print(f'Precision {precision_score(y_gold, baseline)}, Recall {recall_score(y_gold, baseline)}, F1 {f1_score(y_gold, baseline)}')

Precision 0.0, Recall 0.0, F1 0.0


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
# baseline for goldstandard
# flip labels
conf_matrix = confusion_matrix(y_gold, baseline)
precision = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[1,0])
recall = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[0,1])
f1 = (2*precision*recall)/(precision+recall)
print(f'Precision {precision}, Recall {recall}, F1 {f1}')

Precision 0.9640287769784173, Recall 1.0, F1 0.9816849816849816


In [29]:
goldstandard_new.label.value_counts()

0    134
1      5
Name: label, dtype: int64

In [25]:
# fit model
lr.fit(X_train, y_train)
predictions = lr.predict(X_gold)

/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [26]:
# get metrics
print(f'Precision {precision_score(y_gold, predictions)}, Recall {recall_score(y_gold, predictions)}, F1 {f1_score(y_gold, predictions)}')

Precision 0.0967741935483871, Recall 0.6, F1 0.16666666666666666


In [27]:
# flip labels
conf_matrix = confusion_matrix(y_gold, predictions)
precision = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[1,0])
recall = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[0,1])
f1 = (2*precision*recall)/(precision+recall)
print(f'Precision {precision}, Recall {recall}, F1 {f1}')

Precision 0.9814814814814815, Recall 0.7910447761194029, F1 0.8760330578512396


## 1.5 Scaled distance columns + frequencies

In [13]:
lr = LogisticRegression(random_state=42)

In [14]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['id'],
                lr, scale=True)

[[     14 1299996]
 [      1   22845]]
Precision 0.017269649186863727 and Recall 0.9999562286614725 and F1 0.033952917729011274
----------------------------------


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[1299766     244]
 [  22777      69]]
Precision 0.22044728434504793 and Recall 0.00302022235839972 and F1 0.0059588065115074055
----------------------------------
[[1299986      24]
 [  22842       4]]
Precision 0.14285714285714285 and Recall 0.00017508535411012868 and F1 0.0003497420652268952
----------------------------------
[[    124 1299886]
 [      1   22845]]
Precision 0.01727108535295536 and Recall 0.9999562286614725 and F1 0.03395569335682758
----------------------------------
[[1245841   54171]
 [  20792    2057]]
Precision 0.03658319698370918 and Recall 0.09002582169898027 and F1 0.05202524122058247
----------------------------------
---------------------
Overall Precision: 0.0868856717451438
Overall Recall: 0.41862671734688695
Overall F1: 0.025248480176631127
---------------------
Negative Precision: 0.9748856969623592
Negative Recall: 0.5916460770823656
Negative F1: 0.5907019104375375


# 2. Naive Bayes

## 2.1 Distance encoded columns

In [13]:
nb = MultinomialNB()

In [ ]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['frequency', 'pidspread', 
                 'pldspread', 'id'],
                nb)

## 2.2 Scaled Distance encoded columns

In [15]:
nb = MultinomialNB()

In [16]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['frequency', 'pidspread', 
                 'pldspread', 'id'],
                nb, scale=True)

[[1174131  125879]
 [  19492    3354]]
Precision 0.025953123428226536 and Recall 0.1468090694213429 and F1 0.04410865405480046
----------------------------------
[[1155196  144814]
 [  19379    3467]]
Precision 0.023381282834618055 and Recall 0.15175523067495403 and F1 0.040519614087782756
----------------------------------
[[1161300  138710]
 [  19242    3604]]
Precision 0.025324282923675816 and Recall 0.15775190405322595 and F1 0.043642528457253575
----------------------------------
[[1132227  167783]
 [  19015    3831]]
Precision 0.022323353572552357 and Recall 0.16768799789897576 and F1 0.03940141931502623
----------------------------------
[[1182025  117987]
 [  19716    3133]]
Precision 0.025866908850726553 and Recall 0.1371175981443389 and F1 0.04352325847925596
----------------------------------
---------------------
Overall Precision: 0.024569790321959863
Overall Recall: 0.15222436003856749
Overall F1: 0.04223909487882379
---------------------
Negative Precision: 0.98358974348

## 2.4 Scaled Distance columns + frequencies

In [17]:
nb = MultinomialNB()

In [18]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['id'],
                nb,scale=True)

[[1091318  208692]
 [  17694    5152]]
Precision 0.024092328987486204 and Recall 0.22550993609384576 and F1 0.04353373611052431
----------------------------------
[[1062679  237331]
 [  17184    5662]]
Precision 0.023301082747239632 and Recall 0.24783331874288717 and F1 0.042597211093932795
----------------------------------
[[1061095  238915]
 [  17294    5552]]
Precision 0.022710631700802154 and Recall 0.2430184715048586 and F1 0.04153931907539102
----------------------------------
[[1059313  240697]
 [  17354    5492]]
Precision 0.022308064129591493 and Recall 0.24039219119320668 and F1 0.04082740163919192
----------------------------------
[[1061038  238974]
 [  17197    5652]]
Precision 0.023104657722400727 and Recall 0.24736312311260886 and F1 0.04226189363491915
----------------------------------
---------------------
Overall Precision: 0.02310335305750404
Overall Recall: 0.24082340812948141
Overall F1: 0.042151912310791836
---------------------
Negative Precision: 0.98400555387

# 3. XGBoost

## 3.1 Distance columns

In [19]:
xg = xgb.XGBRFClassifier(random_state=42)

In [20]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['frequency', 'pidspread', 
                 'pldspread', 'id'],
                xg)

[[898411 401599]
 [  8709  14137]]
Precision 0.03400475301633729 and Recall 0.6187954127637223 and F1 0.06446684998472349
----------------------------------
[[883851 416159]
 [  8545  14301]]
Precision 0.03322259908005389 and Recall 0.6259739122822376 and F1 0.06309645140368757
----------------------------------
[[997255 302755]
 [ 11753  11093]]
Precision 0.035345135224694756 and Recall 0.4855554582859144 and F1 0.06589366011868343
----------------------------------
[[898275 401735]
 [  8634  14212]]
Precision 0.03416781464946255 and Recall 0.6220782631532872 and F1 0.06477769700063585
----------------------------------
[[990501 309511]
 [ 11417  11432]]
Precision 0.035620032217558883 and Recall 0.5003282419361897 and F1 0.06650532880346255
----------------------------------
---------------------
Overall Precision: 0.03447206683762148
Overall Recall: 0.5705462576842704
Overall F1: 0.06494799746223857
---------------------
Negative Precision: 0.9896520944728501
Negative Recall: 0.71819

## 3.2 Scaled Distance columns

In [21]:
xg = xgb.XGBRFClassifier(random_state=42)

In [22]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['frequency', 'pidspread', 
                 'pldspread', 'id'],
                xg, scale=True)

[[      1 1300009]
 [      0   22846]]
Precision 0.017270222359971426 and Recall 1.0 and F1 0.033954050714088795
----------------------------------
[[1297012    2998]
 [  22506     340]]
Precision 0.10185739964050329 and Recall 0.014882255099360938 and F1 0.025970058050717997
----------------------------------
[[1300006       4]
 [  22846       0]]
Precision 0.0 and Recall 0.0 and F1 0.0
----------------------------------
[[    104 1299906]
 [      0   22846]]
Precision 0.017271567156957615 and Recall 1.0 and F1 0.033956649757208315
----------------------------------
[[    250 1299762]
 [      2   22847]]
Precision 0.017274190633815436 and Recall 0.999912468817016 and F1 0.03396166955787546
----------------------------------
---------------------
Overall Precision: 0.030734675958249552
Overall Recall: 0.6029589447832755
Overall F1: 0.025568485615978114
---------------------
Negative Precision: 0.9915474004860203
Negative Recall: 0.39959277230411894
Negative F1: 0.39641961981545637


## 3.3 Distance columns + frequencies

In [23]:
xg = xgb.XGBRFClassifier(random_state=42)

In [24]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['id'],
                xg)

[[901319 398691]
 [  8732  14114]]
Precision 0.03419047734402442 and Recall 0.617788671977589 and F1 0.06479498497650642
----------------------------------
[[884840 415170]
 [  8566  14280]]
Precision 0.033251833740831294 and Recall 0.6250547141731594 and F1 0.06314448944938712
----------------------------------
[[997766 302244]
 [ 11753  11093]]
Precision 0.03540277720154339 and Recall 0.4855554582859144 and F1 0.06599381884271364
----------------------------------
[[897208 402802]
 [  8613  14233]]
Precision 0.03412902993753522 and Recall 0.6229974612623654 and F1 0.0647129564586786
----------------------------------
[[989561 310451]
 [ 11406  11443]]
Precision 0.035548969536555515 and Recall 0.5008096634426015 and F1 0.06638568440838538
----------------------------------
---------------------
Overall Precision: 0.034504617552097966
Overall Recall: 0.5704411938283259
Overall F1: 0.06500638682713424
---------------------
Negative Precision: 0.9896542494027262
Negative Recall: 0.718562

## 3.4 Scaled Distance columns + frequencies

In [25]:
xg = xgb.XGBRFClassifier(random_state=42)

In [26]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['id'],
                xg, scale=True)

[[      1 1300009]
 [      0   22846]]
Precision 0.017270222359971426 and Recall 1.0 and F1 0.033954050714088795
----------------------------------
[[1294153    5857]
 [  22361     485]]
Precision 0.07647429832860296 and Recall 0.021229099185853105 and F1 0.03323283541181308
----------------------------------
[[1299982      28]
 [  22840       6]]
Precision 0.17647058823529413 and Recall 0.00026262803116519303 and F1 0.0005244755244755245
----------------------------------
[[ 233603 1066407]
 [   2387   20459]]
Precision 0.018823847650032293 and Recall 0.8955178149347807 and F1 0.036872630015715786
----------------------------------
[[ 294543 1005469]
 [   5870   16979]]
Precision 0.016606223494984584 and Recall 0.7430959779421419 and F1 0.03248646078578624
----------------------------------
---------------------
Overall Precision: 0.06112903601377708
Overall Recall: 0.5320211040187882
Overall F1: 0.027414090490375888
---------------------
Negative Precision: 0.9872188545784791
Negativ

# 4. Random Forest

## 4.1 Distance encoded columns

In [27]:
rf = RandomForestClassifier(random_state=42)

In [28]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['frequency', 'pidspread', 
                 'pldspread', 'id'],
                rf)

[[851705 448305]
 [  7402  15444]]
Precision 0.03330249768732655 and Recall 0.6760045522192069 and F1 0.06347784091492926
----------------------------------
[[838241 461769]
 [  7196  15650]]
Precision 0.03278042976923834 and Recall 0.6850214479558785 and F1 0.06256683957502523
----------------------------------
[[986999 313011]
 [ 11193  11653]]
Precision 0.03589249193011852 and Recall 0.5100674078613324 and F1 0.06706569595119564
----------------------------------
[[839449 460561]
 [  7029  15817]]
Precision 0.03320262480635126 and Recall 0.6923312614899764 and F1 0.06336634456676762
----------------------------------
[[970313 329699]
 [ 10628  12221]]
Precision 0.03574227889564811 and Recall 0.5348592936233533 and F1 0.06700679059898183
----------------------------------
---------------------
Overall Precision: 0.034184064617736556
Overall Recall: 0.6196567926299494
Overall F1: 0.06469670232137992
---------------------
Negative Precision: 0.9905041830239313
Negative Recall: 0.690257

## 4.2 Scaled Distance encoded columns

In [29]:
rf = RandomForestClassifier(random_state=42)

In [30]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['frequency', 'pidspread', 
                 'pldspread', 'id'],
                rf, scale=True)

[[1006575  293435]
 [  16642    6204]]
Precision 0.020704914914280184 and Recall 0.2715573842248096 and F1 0.03847620819573004
----------------------------------
[[1279753   20257]
 [  21907     939]]
Precision 0.04430081147386299 and Recall 0.04110128687735271 and F1 0.04264111529903274
----------------------------------
[[  78650 1221360]
 [   2237   20609]]
Precision 0.016593811922841876 and Recall 0.9020835157139105 and F1 0.03258816506761859
----------------------------------
[[1178019  121991]
 [  18532    4314]]
Precision 0.0341554174419065 and Recall 0.1888295544077738 and F1 0.05784741637669208
----------------------------------
[[   5761 1294251]
 [    106   22743]]
Precision 0.017268871384379884 and Recall 0.9953608473018513 and F1 0.033948753697261545
----------------------------------
---------------------
Overall Precision: 0.026604765427454286
Overall Recall: 0.4797865177051396
Overall F1: 0.041100331727267
---------------------
Negative Precision: 0.9811389387388839
Neg

## 4.3 Distance columns + frequencies

In [31]:
rf = RandomForestClassifier(random_state=42)

In [32]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['id'],
                rf)

[[864743 435267]
 [  7473  15373]]
Precision 0.0341137049529558 and Recall 0.6728967871837521 and F1 0.06493539407712161
----------------------------------
[[871817 428193]
 [  7717  15129]]
Precision 0.034126436314913314 and Recall 0.6622165805830342 and F1 0.06490793018825831
----------------------------------
[[919065 380945]
 [  9938  12908]]
Precision 0.03277364905180359 and Recall 0.5650004377133853 and F1 0.06195359240122967
----------------------------------
[[867755 432255]
 [  7461  15385]]
Precision 0.03436913591278706 and Recall 0.6734220432460825 and F1 0.06540045824955472
----------------------------------
[[942581 357431]
 [ 10058  12791]]
Precision 0.034549540545942704 and Recall 0.5598056807737757 and F1 0.0650823896955003
----------------------------------
---------------------
Overall Precision: 0.0339864933556805
Overall Recall: 0.626668305900006
Overall F1: 0.06445595292233292
---------------------
Negative Precision: 0.9905755844126652
Negative Recall: 0.687065414

### 4.4 scaled Distance columns + frequencies

In [33]:
rf = RandomForestClassifier(random_state=42)

In [34]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['id'],
                rf, scale=True)

[[1139108  160902]
 [  18906    3940]]
Precision 0.023901675543854115 and Recall 0.17245907379847675 and F1 0.041984570137675294
----------------------------------
[[1296018    3992]
 [  22521     325]]
Precision 0.07528376187167014 and Recall 0.014225685021447956 and F1 0.02392961013142878
----------------------------------
[[1185234  114776]
 [  20362    2484]]
Precision 0.02118369435442606 and Recall 0.10872800490238992 and F1 0.035458866857950404
----------------------------------
[[1276307   23703]
 [  21879     967]]
Precision 0.03919740575597892 and Recall 0.04232688435612361 and F1 0.04070207929960435
----------------------------------
[[917728 382284]
 [ 14105   8744]]
Precision 0.022361570015446464 and Recall 0.38268633200577706 and F1 0.04225409964796304
----------------------------------
---------------------
Overall Precision: 0.036385621508275144
Overall Recall: 0.14408519601684305
Overall F1: 0.03686584521492437
---------------------
Negative Precision: 0.983542716578977

# 5. MLPClassifier

## 5.1 Distance encoded columns

In [35]:
nnet = MLPClassifier(random_state=42)

In [36]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['frequency', 'pidspread', 
                 'pldspread', 'id'],
                nnet)

[[802202 497808]
 [  6753  16093]]
Precision 0.03131537008100782 and Recall 0.7044121509235752 and F1 0.05996493692559064
----------------------------------
[[854192 445818]
 [  7673  15173]]
Precision 0.03291387467434288 and Recall 0.6641425194782457 and F1 0.06271946957343072
----------------------------------
[[968774 331236]
 [ 10884  11962]]
Precision 0.034854515469204365 and Recall 0.5235927514663399 and F1 0.0653582629410672
----------------------------------
[[869406 430604]
 [  7593  15253]]
Precision 0.03421052041349581 and Recall 0.6676442265604482 and F1 0.06508599262219358
----------------------------------
[[944846 355166]
 [ 10149  12700]]
Precision 0.0345234406006535 and Recall 0.5558230119480064 and F1 0.06500902192134932
----------------------------------
---------------------
Overall Precision: 0.03356354424774087
Overall Recall: 0.623122932075323
Overall F1: 0.0636275367967263
---------------------
Negative Precision: 0.9904708383727081
Negative Recall: 0.6829822149

## 5.2 Scaled Distance encoded columns

In [37]:
nnet = MLPClassifier(random_state=42)

In [38]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['frequency', 'pidspread', 
                 'pldspread', 'id'],
                nnet, scale=True)

[[   4893 1295117]
 [     26   22820]]
Precision 0.01731493994022476 and Recall 0.9988619451982842 and F1 0.0340398110656236
----------------------------------
[[1201880   98130]
 [  19933    2913]]
Precision 0.02882931029363736 and Recall 0.12750590913070123 and F1 0.047025966792854895
----------------------------------
[[1247745   52265]
 [  20783    2063]]
Precision 0.03797305256957738 and Recall 0.09030027138229887 and F1 0.053463601731152985
----------------------------------
[[ 143113 1156897]
 [   1187   21659]]
Precision 0.018377573912482733 and Recall 0.9480434211678194 and F1 0.03605620766404584
----------------------------------
[[ 190180 1109832]
 [   1924   20925]]
Precision 0.01850530220020747 and Recall 0.9157950019694516 and F1 0.03627755056752478
----------------------------------
---------------------
Overall Precision: 0.024200035783225944
Overall Recall: 0.6161013097697111
Overall F1: 0.04137262756424041
---------------------
Negative Precision: 0.9887550418840085
N

## 5.3 Distance encoded columns + frequencies

In [39]:
nnet = MLPClassifier(random_state=42)

In [40]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['id'],
                nnet)

[[875760 424250]
 [  7515  15331]]
Precision 0.03487639365668671 and Recall 0.6710583909655957 and F1 0.0663066819195246
----------------------------------
[[850948 449062]
 [  7186  15660]]
Precision 0.0336975654262118 and Recall 0.6854591613411538 and F1 0.06423719358120304
----------------------------------
[[827044 472966]
 [  8335  14511]]
Precision 0.02976755826428734 and Recall 0.6351658933730193 and F1 0.05686986477191896
----------------------------------
[[851823 448187]
 [  7012  15834]]
Precision 0.0341234556194655 and Recall 0.6930753742449444 and F1 0.06504445772664813
----------------------------------
[[974576 325436]
 [ 10536  12313]]
Precision 0.036456066487243484 and Recall 0.5388857280406145 and F1 0.06829211476491827
----------------------------------
---------------------
Overall Precision: 0.03378420789077897
Overall Recall: 0.6447289095930655
Overall F1: 0.0641500625528426
---------------------
Negative Precision: 0.9908561242256049
Negative Recall: 0.6738639703

## 5.4 Scaled Distance encoded columns + frequencies

In [41]:
nnet = MLPClassifier(random_state=42)

In [42]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['id'],
                nnet, scale=True)

/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[[    869 1299141]
 [      7   22839]]
Precision 0.017276358189987746 and Recall 0.9996936006303073 and F1 0.03396573236983818
----------------------------------
[[1172248  127762]
 [  18143    4703]]
Precision 0.03550371796323557 and Recall 0.2058566050949838 and F1 0.06056235553180393
----------------------------------
[[1292032    7978]
 [  22114     732]]
Precision 0.08404133180252583 and Recall 0.03204061980215355 and F1 0.046393712764608956
----------------------------------
[[ 273964 1026046]
 [   2894   19952]]
Precision 0.01907460626119744 and Recall 0.8733257463013219 and F1 0.03733379239627111
----------------------------------
[[313398 986614]
 [  3569  19280]]
Precision 0.01916702952796219 and Recall 0.8438006039651625 and F1 0.03748263657687099
----------------------------------
---------------------
Overall Precision: 0.03501260874898175
Overall Recall: 0.5909434351587859
Overall F1: 0.04314764592787863
---------------------
Negative Precision: 0.9876454808175177
Negativ

# 6. DecisionTreeClassifier

## 6.1 Distance encoded columns

In [43]:
dt = DecisionTreeClassifier(random_state=42)

In [44]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['frequency', 'pidspread', 
                 'pldspread', 'id'],
                dt)

[[836303 463707]
 [  8236  14610]]
Precision 0.030544596993207433 and Recall 0.639499255887245 and F1 0.05830438400280947
----------------------------------
[[824983 475027]
 [  8173  14673]]
Precision 0.029963242801715338 and Recall 0.6422568502144795 and F1 0.05725534878820633
----------------------------------
[[977660 322350]
 [ 11827  11019]]
Precision 0.0330534632794291 and Recall 0.482316379234877 and F1 0.061867130805833545
----------------------------------
[[825363 474647]
 [  7866  14980]]
Precision 0.030594718020043828 and Recall 0.655694651142432 and F1 0.058461616514431
----------------------------------
[[962668 337344]
 [ 11349  11500]]
Precision 0.032966024928048066 and Recall 0.5033043021576437 and F1 0.06187902381804338
----------------------------------
---------------------
Overall Precision: 0.03142440920448875
Overall Recall: 0.5846142877273355
Overall F1: 0.059553500785864744
---------------------
Negative Precision: 0.9894786917989178
Negative Recall: 0.6810679

## 6.2 Scaled Distance encoded columns

In [45]:
dt = DecisionTreeClassifier(random_state=42)

In [46]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['frequency', 'pidspread', 
                 'pldspread', 'id'],
                dt, scale=True)

[[742312 557698]
 [ 13478   9368]]
Precision 0.016520122878112952 and Recall 0.4100498993259214 and F1 0.03176066938797651
----------------------------------
[[742323 557687]
 [ 14546   8300]]
Precision 0.01466464777459553 and Recall 0.363302109778517 and F1 0.02819135476442387
----------------------------------
[[1190740  109270]
 [  19822    3024]]
Precision 0.026929310559780577 and Recall 0.1323645277072573 and F1 0.04475358887080065
----------------------------------
[[1261862   38148]
 [  21705    1141]]
Precision 0.02904120746264858 and Recall 0.04994309725991421 and F1 0.03672648265872697
----------------------------------
[[  46089 1253923]
 [   1497   21352]]
Precision 0.01674305541941934 and Recall 0.9344829095365224 and F1 0.03289670324252537
----------------------------------
---------------------
Overall Precision: 0.020779668818911395
Overall Recall: 0.37802850872162647
Overall F1: 0.03486575978489067
---------------------
Negative Precision: 0.9796410802217801
Negative R

## 6.3 Only distance encoded columns + frequencies

In [47]:
dt = DecisionTreeClassifier(random_state=42)

In [48]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['id'],
                dt)

[[849722 450288]
 [  8753  14093]]
Precision 0.030347925518055218 and Recall 0.6168694738685109 and F1 0.05784983180324572
----------------------------------
[[848365 451645]
 [  8771  14075]]
Precision 0.030222021815683245 and Recall 0.6160815897750154 and F1 0.05761759925987483
----------------------------------
[[911338 388672]
 [ 10959  11887]]
Precision 0.029676027751217672 and Recall 0.5203099010767749 and F1 0.05614954948571698
----------------------------------
[[845918 454092]
 [  8565  14281]]
Precision 0.030490655951559975 and Recall 0.6250984855116869 and F1 0.05814514503714229
----------------------------------
[[932878 367134]
 [ 11065  11784]]
Precision 0.031099076845122163 and Recall 0.5157337301413628 and F1 0.058660865626096716
----------------------------------
---------------------
Overall Precision: 0.030367141576327656
Overall Recall: 0.5788186360746702
Overall F1: 0.05768459824241531
---------------------
Negative Precision: 0.9891886194340745
Negative Recall: 0.

## 6.4 Scaled distance encoded columns + frequencies

In [49]:
dt = DecisionTreeClassifier(random_state=42)

In [50]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['id'],
                dt, scale=True)

[[1156637  143373]
 [  20043    2803]]
Precision 0.01917551444833625 and Recall 0.12269106189267268 and F1 0.03316727999905337
----------------------------------
[[ 282734 1017276]
 [   4779   18067]]
Precision 0.01745025561577178 and Recall 0.7908167731769238 and F1 0.03414701910528271
----------------------------------
[[1193674  106336]
 [  20078    2768]]
Precision 0.025370288898665494 and Recall 0.12115906504420905 and F1 0.04195528609321713
----------------------------------
[[1228732   71278]
 [  21505    1341]]
Precision 0.018466241617207616 and Recall 0.05869736496542064 and F1 0.028094065888021783
----------------------------------
[[  56991 1243021]
 [   1272   21577]]
Precision 0.017062339178141987 and Recall 0.9443301676222154 and F1 0.03351904971622133
----------------------------------
---------------------
Overall Precision: 0.019504927951624625
Overall Recall: 0.4075388865402883
Overall F1: 0.03417654016035926
---------------------
Negative Precision: 0.982153950875213